In [1]:
import pandas as pd
import numpy as np
from pyeeg import (
    bin_power,
    hfd,
    hjorth,
    spectral_entropy,
    svd_entropy,
    fisher_info,
    ap_entropy,
    dfa,
    hurst,
    permutation_entropy,
    samp_entropy,
    pfd
)

In [2]:
df = pd.read_csv('timeseries_NEW.csv')

In [3]:
df.drop(df.columns[0], axis=1, inplace=True)

In [4]:
df.head()

,s0209110,s0310211,s0310411,s0310703,s0310706,s0310710,s0311303,s0320307,s0408108,s0410108,...,s8010707,s8108205,s8420304,s8508108,s8608101,s8710205,s9309101,s9509011,s9709110,s9809003
0,8.32,0.0,26.75,0.0,72.66,74.72,0.0,0.13,12.36,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0
1,5.57,0.0,30.88,0.0,33.76,120.70,0.0,0.06,13.88,0.0,...,0.00,0.2,0.00,0.0,4.75,0.0,5.35,0.0,0.0,0.0
2,13.60,0.0,17.23,0.0,38.43,199.51,0.0,29.35,10.79,0.1,...,6.34,0.0,0.03,0.0,0.00,0.0,0.47,0.0,0.0,0.0
3,5.06,0.0,1.75,0.0,21.80,60.94,0.0,72.06,27.75,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0
4,0.00,0.0,11.72,0.0,26.64,35.22,0.0,39.49,50.18,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0


In [5]:
df.dtypes.unique()

array([dtype('float64')], dtype=object)

In [6]:
df_features = pd.DataFrame()

In [7]:
df_features.index = df.columns

In [8]:
df_features['mean'] = df.mean()
df_features['max'] = df.max()
df_features['var'] = df.var()
quantile = df.quantile([0.25, 0.5, 0.75]).T
df_features[quantile.columns] = quantile

In [9]:
days = np.split(df.to_numpy(),7)
day_mean = np.array([day.mean(axis=0) for day in days])
df_features['day_var'] = day_mean.var(axis=0)

In [10]:
hjorth = np.array([hjorth(df[column]) for column in df])
df_features['hjorth_mobility'] = hjorth[:,0]
df_features['hjorth_complexity'] = hjorth[:,1]

In [11]:
df_features['dfa'] = [dfa(df[column]) for column in df]

c:\Users\alexj\actigraphy-classification\.env\lib\site-packages\pyeeg-0.4.4-py3.10.egg\pyeeg\detrended_fluctuation_analysis.py:125: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
c:\Users\alexj\actigraphy-classification\.env\lib\site-packages\pyeeg-0.4.4-py3.10.egg\pyeeg\detrended_fluctuation_analysis.py:129: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


In [12]:
df_features['pfd'] = [pfd(df[column]) for column in df]

In [13]:
df_features['permutation_entropy_n_3'] = [permutation_entropy(df[column], 3, 1) for column in df]
df_features['permutation_entropy_n_4'] = [permutation_entropy(df[column], 4, 1) for column in df]

In [14]:
df_features['spectral_entropy'] = [spectral_entropy(df[column], np.arange(0, 10080, 24*7), 1) for column in df]

c:\Users\alexj\actigraphy-classification\.env\lib\site-packages\pyeeg-0.4.4-py3.10.egg\pyeeg\entropy.py:236: RuntimeWarning: divide by zero encountered in log
c:\Users\alexj\actigraphy-classification\.env\lib\site-packages\pyeeg-0.4.4-py3.10.egg\pyeeg\entropy.py:236: RuntimeWarning: invalid value encountered in double_scalars


In [15]:
df_features['spectral_entropy'].isna().sum()

96

In [16]:
def heaviside_filter(X, f, filter_value):
    return np.heaviside(X - f*filter_value,1).sum()

In [17]:
df_features['above_0.7*max'] = heaviside_filter(df, 0.7, df.max())
df_features['above_0.8*max'] = heaviside_filter(df, 0.8, df.max())
df_features['above_0.9*max'] = heaviside_filter(df, 0.9, df.max())
df_features['above_0.7*mean'] = heaviside_filter(df, 0.7, df.mean())
df_features['above_0.8*mean'] = heaviside_filter(df, 0.8, df.mean())
df_features['above_0.9*mean'] = heaviside_filter(df, 0.9, df.mean())

In [18]:
df_features.head()

,mean,max,var,0.25,0.5,0.75,day_var,hjorth_mobility,hjorth_complexity,dfa,pfd,permutation_entropy_n_3,permutation_entropy_n_4,spectral_entropy,above_0.7*max,above_0.8*max,above_0.9*max,above_0.7*mean,above_0.8*mean,above_0.9*mean
s0209110,32.103135,503.85,2247.832675,0.00,12.030,45.6925,39.145362,0.005811,274.105198,0.965812,1.018695,2.433537,4.217144,NaN,9.0,5.0,3.0,3972.0,3733.0,3493.0
s0310211,32.910309,652.34,3116.790385,0.00,10.645,46.1525,183.527628,0.005470,295.670221,1.009446,1.019075,2.445443,4.237246,NaN,15.0,5.0,1.0,3787.0,3580.0,3362.0
s0310411,35.884296,250.16,1627.644029,0.48,19.295,63.6100,31.404157,0.004533,352.741354,1.016422,1.019957,2.477711,4.309999,NaN,39.0,13.0,3.0,4572.0,4287.0,4036.0
s0310703,32.294832,599.57,3006.353329,0.00,6.620,46.0425,127.994579,0.005615,283.448745,0.985328,1.017197,2.363372,4.055431,NaN,26.0,11.0,5.0,3631.0,3423.0,3245.0
s0310706,27.823337,378.21,1319.747447,0.00,12.555,44.9825,27.663564,0.005501,293.167548,0.997671,1.017338,2.368780,4.064085,NaN,5.0,3.0,2.0,4382.0,4126.0,3913.0
